In [226]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, make_scorer

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

Todas as informações da coluna Laptop já estão definidas na outras colunas. Então não será necessário NLP nesse processo

In [175]:
df = pd.read_csv('atividade-1-precificacao-laptops.csv')
with pd.option_context('display.max_colwidth', None):  # more options can be specified also
    display(df.head())

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,"ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core i5-1235U/8GB/512GB SSD/15.6""",New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,"Alurin Go Start Intel Celeron N4020/8GB/256GB SSD/15.6""",New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,"ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core i3-1215U/8GB/256GB SSD/15.6""",New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,"MSI Katana GF66 12UC-082XES Intel Core i7-12700H/16GB/1TB SSD/RTX3050/15.6""",New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,"HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB SSD/15.6""",New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


Visto também que as colunas já estão com os tipos corretamente definidos.

In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Laptop        2160 non-null   object 
 1   Status        2160 non-null   object 
 2   Brand         2160 non-null   object 
 3   Model         2160 non-null   object 
 4   CPU           2160 non-null   object 
 5   RAM           2160 non-null   int64  
 6   Storage       2160 non-null   int64  
 7   Storage type  2118 non-null   object 
 8   GPU           789 non-null    object 
 9   Screen        2156 non-null   float64
 10  Touch         2160 non-null   object 
 11  Final Price   2160 non-null   float64
dtypes: float64(2), int64(2), object(8)
memory usage: 202.6+ KB


Temos NaN values

In [177]:
temp = df.isna().sum()
temp[temp>0]

Storage type      42
GPU             1371
Screen             4
dtype: int64

GPU com NaN, entende-se que são notebooks sem GPU

In [178]:
df[df['GPU'].isna()]

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01
6,Lenovo Thinkpad E14 Gen 4 AMD Ryzen 5 5625U/8G...,New,Lenovo,ThinkPad,AMD Ryzen 5,8,256,SSD,NaN,14.0,No,909.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2136,Microsoft Surface Pro 7 Intel Core i7-1065G7/1...,Refurbished,Microsoft,Surface Pro,Intel Core i7,16,256,SSD,NaN,12.3,No,1094.03
2137,Microsoft Surface Pro X Microsoft SQ1/8 GB/128...,Refurbished,Microsoft,Surface Pro,Microsoft SQ1,8,128,SSD,NaN,13.0,Yes,1049.00
2138,Microsoft Surface Pro X Microsoft SQ1/8 GB/256...,Refurbished,Microsoft,Surface Pro,Microsoft SQ1,8,256,SSD,NaN,13.0,Yes,1299.00
2158,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,Refurbished,Razer,Book,Intel Evo Core i7,16,1000,SSD,NaN,13.4,Yes,1899.99


In [179]:
df['GPU'] = df['GPU'].fillna('None')
df[df['GPU'].isna()]

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price


Screen com NaN não faz sentido, e como são poucas linhas, um drop não irá ser relevante

In [180]:
df[df['Screen'].isna()]

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
624,Acer Extensa 15 EX215-54 Intel Core i5-1135G7/...,New,Acer,Extensa,Intel Core i5,8,256,SSD,None,NaN,No,524.99
1430,HP ENVY x360 2-in-1 Laptop 15-ew0008np Intel C...,New,HP,Envy,Intel Core i7,16,512,SSD,RTX 2050,NaN,Yes,1863.52
1503,Lenovo IdeaPad Gaming 3 15ACH6 AMD Ryzen 5 560...,New,Lenovo,IdeaPad,AMD Ryzen 5,16,512,SSD,RTX 3060,NaN,No,1505.00
1548,Lenovo ThinkPad P15 Gen 2 Intel Core i7-11850H...,New,Lenovo,ThinkPad,Intel Core i7,16,512,SSD,RTX A2000,NaN,No,2569.00


In [181]:
df = df.dropna(subset='Screen').reset_index(drop=True)
df[df['Screen'].isna()]

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price


Quantidade de Storage type com NaN é significativa, então será outra categoria

In [182]:
print(df[df['Storage type'].isna()].shape)
df[df['Storage type'].isna()].head()

(42, 12)


,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
145,ASUS Chromebook CX1400CNA-BV0210 Intel Celeron...,New,Asus,Chromebook,Intel Celeron,8,32,NaN,None,14.0,No,349.00
189,Portátil Alurin Flex Advance Intel Core I5-115...,Refurbished,Alurin,Flex Advance,Intel Core i5,8,0,NaN,None,14.0,No,368.70
260,ASUS ROG Strix G16 G614JZ-N3008 Intel Core i7-...,New,Asus,ROG,Intel Core i7,32,1000,NaN,RTX 4080,16.0,No,2599.01
477,Prixton Flex Pro Intel Celeron N4020/4GB/64GB/...,New,Prixton,Flex,Intel Celeron,4,64,NaN,None,11.6,Yes,329.95
493,"Apple MacBook Pro Intel Core i5/8GB/256GB/13"" ...",Refurbished,Apple,MacBook Pro,Intel Core i5,8,256,NaN,None,13.0,No,634.25


In [183]:
df['Storage type'] = df['Storage type'].fillna('None')
df[df['Storage type'].isna()].head()

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price


In [184]:
df = df.reset_index(drop=True)

In [185]:
print(f'Marcas diferentes: {df.Brand.nunique()}')
print(df['Brand'].value_counts())

Marcas diferentes: 27
Brand
Asus                415
HP                  367
Lenovo              364
MSI                 308
Acer                136
Apple               116
Dell                 84
Microsoft            77
Gigabyte             48
Razer                37
Medion               32
LG                   32
Alurin               29
PcCom                24
Samsung              22
Dynabook Toshiba     19
Vant                 11
Primux                8
Deep Gaming           8
Innjoo                6
Thomson               4
Prixton               3
Millenium             2
Denver                1
Jetwing               1
Realme                1
Toshiba               1
Name: count, dtype: int64


In [186]:
df['Status'].value_counts()

Status
New            1494
Refurbished     662
Name: count, dtype: int64

In [187]:
print(df['CPU'].nunique())
df['CPU'].value_counts()

28


CPU
Intel Core i7            708
Intel Core i5            534
AMD Ryzen 7              156
Intel Core i3            130
AMD Ryzen 5              126
Intel Core i9             94
Intel Celeron             94
Intel Evo Core i7         82
AMD Ryzen 9               44
AMD Ryzen 3               44
Intel Evo Core i5         30
Apple M2                  28
AMD 3020e                 13
Apple M2 Pro              13
Apple M1                  11
AMD Athlon                10
Intel Pentium             10
Apple M1 Pro               7
Intel Core M3              5
AMD 3015e                  3
Microsoft SQ1              3
Qualcomm Snapdragon 7      3
AMD Radeon 9               2
Qualcomm Snapdragon 8      2
Intel Evo Core i9          1
Mediatek MT8183            1
AMD 3015Ce                 1
AMD Radeon 5               1
Name: count, dtype: int64

Considerando que o modelo Tecra está em Toshiba e Dynabook Toshiba, renomearei para ter uma marca apenas

In [188]:
df['Brand'] = df['Brand'].replace('Toshiba', 'Dynabook Toshiba')

Dropar marcas com menos de 3

In [189]:
df = df.groupby('Brand').filter(lambda group: len(group)>3).reset_index(drop=True)

In [190]:
df['GPU'].unique()

array(['None', 'RTX 3050', 'RTX 4060', 'RTX 4050', 'RTX 3060', 'RTX 4070',
       'RTX 2050', 'GTX 1650', 'RTX 3070', '610 M', 'RTX 4080',
       'RX 6500M', 'MX 550', 'RTX 3080', 'RTX 4090', 'RX 7600S', 'A 370M',
       'GTX 1660', 'RTX A1000', 'RTX 3000', 'T 1200', 'Radeon Pro 5300M',
       'A 730M', 'Radeon Pro RX 560X', 'RTX A5500', 'Radeon Pro 5500M',
       'T 500', 'T 550', 'RTX A3000', 'T 2000', 'T 600', 'T 1000',
       'Radeon RX 6600M', 'MX 330', 'RTX A2000', 'MX 450', 'RTX 2070',
       'RX 6800S', 'RTX 2080', 'RTX 2060', 'GTX 1050', 'MX 130', 'P 500',
       'RX 6700M', 'GTX 1070'], dtype=object)

In [191]:
px.scatter(df, x='GPU', y='CPU', color='Final Price')

"compact" the different types of GPUs to RTX/GTX/MX/T/AMD/None

In [192]:
def map_gpu_category(gpu_type):
    if gpu_type == 'None':
        return 'None'
    elif 'RTX' in gpu_type:
        return 'RTX'
    elif 'GTX' in gpu_type:
        return 'GTX'
    elif 'MX' in gpu_type:
        return 'MX'
    elif 'T' in gpu_type:
        return 'T'
    elif 'Radeon' in gpu_type or 'RX' in gpu_type:
        return 'AMD'
    else:
        return 'Other'

# Apply the mapping function to create a new column 'gpu_category'
df['GPU'] = df['GPU'].apply(map_gpu_category)

print(df['GPU'].value_counts())
px.scatter(df, x='GPU', y='Final Price', color='Final Price')

GPU
None     1364
RTX       653
GTX        67
AMD        22
T          19
MX         18
Other       5
Name: count, dtype: int64


In [193]:
px.scatter(df, x='Brand', y='Final Price', color='Model')

In [194]:
print(df[['Storage', 'Final Price']].corr())
px.scatter(df, x='Storage', y='Final Price', color='Storage type')

              Storage  Final Price
Storage      1.000000     0.695679
Final Price  0.695679     1.000000


In [195]:
print(df[['RAM', 'Final Price']].corr())
px.scatter(df, x='RAM', y='Final Price', color='Storage')

                  RAM  Final Price
RAM          1.000000     0.724639
Final Price  0.724639     1.000000


In [196]:
print(df[['Screen', 'Final Price']].corr())
px.scatter(df, x='Screen', y='Final Price', color='Storage')

               Screen  Final Price
Screen       1.000000     0.267072
Final Price  0.267072     1.000000


In [197]:
# notebooks com e sem touch englobam quase todo o range, pode ser que confunda o modelo
px.scatter(df, x='Touch', y='Final Price')

One Hot Encoding

In [198]:
categorical_columns = ['Status', 'Brand', 'Model', 'CPU', 'Storage type', 'GPU', 'Touch']
numerical_columns = ['RAM', 'Storage', 'Screen']
target = 'Final Price'

df_encoded = pd.DataFrame()
# df_encoded[target] = df[target]
df_encoded[numerical_columns] = df[numerical_columns]

print(df_encoded.shape)
df_encoded.head()

(2148, 3)


,RAM,Storage,Screen
0,8,512,15.6
1,8,256,15.6
2,8,256,15.6
3,16,1000,15.6
4,16,512,15.6


In [199]:
# Inicializador OneHotEncoder
encoder = OneHotEncoder()

# Initializador ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[('cat', encoder, categorical_columns)]
)

# Aplicando as transformações para ter a sparse matrix
transformed_data = preprocessor.fit_transform(df)

# Convertendo a sparse matrix para um array denso
transformed_array = transformed_data.toarray()

# Novos nomes de colunas após o one hot
new_column_names = preprocessor.named_transformers_['cat'].get_feature_names_out(input_features=categorical_columns)

# Voltando para um dataframe
transformed_df = pd.DataFrame(transformed_array, columns=new_column_names.tolist())

print(transformed_df.shape)
transformed_df.head()


(2148, 178)


,Status_New,Status_Refurbished,Brand_Acer,Brand_Alurin,Brand_Apple,Brand_Asus,Brand_Deep Gaming,Brand_Dell,Brand_Dynabook Toshiba,Brand_Gigabyte,...,Storage type_eMMC,GPU_AMD,GPU_GTX,GPU_MX,GPU_None,GPU_Other,GPU_RTX,GPU_T,Touch_No,Touch_Yes
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [200]:
# concatenand com o restante das covariáveis
df_encoded = pd.concat([df_encoded, transformed_df], axis=1)

print(df_encoded.shape)
df_encoded.head()

(2148, 181)


,RAM,Storage,Screen,Status_New,Status_Refurbished,Brand_Acer,Brand_Alurin,Brand_Apple,Brand_Asus,Brand_Deep Gaming,...,Storage type_eMMC,GPU_AMD,GPU_GTX,GPU_MX,GPU_None,GPU_Other,GPU_RTX,GPU_T,Touch_No,Touch_Yes
0,8,512,15.6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,8,256,15.6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,8,256,15.6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,16,1000,15.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,16,512,15.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


Features e target para X e Y e train test split

In [201]:
X = df_encoded
y = df['Final Price']

features = df_encoded.columns
target = df_encoded.columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4712389)
print(X_train.shape)
X_train.head()

(1718, 181)


,RAM,Storage,Screen,Status_New,Status_Refurbished,Brand_Acer,Brand_Alurin,Brand_Apple,Brand_Asus,Brand_Deep Gaming,...,Storage type_eMMC,GPU_AMD,GPU_GTX,GPU_MX,GPU_None,GPU_Other,GPU_RTX,GPU_T,Touch_No,Touch_Yes
270,8,256,15.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1817,16,512,14.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1865,8,512,15.6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1246,8,512,15.6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
868,8,512,16.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


MinMaxScaler fit no dataset de treino apenas

In [202]:
# min_max_scaler = MinMaxScaler()

In [203]:
# X_train[numerical_columns] = min_max_scaler.fit_transform(X_train[numerical_columns])
# X_test[numerical_columns] = min_max_scaler.transform(X_test[numerical_columns])
# X_train.head()

Criação de um dataframe com as métricas para avaliar o melhor modelo

In [204]:
df_metrics = pd.DataFrame(columns=['MSE', 'MAE', 'R2'])

def create_metrics(model, y_test, y_pred):
	mse = mean_squared_error(y_test, y_pred)
	r2 = r2_score(y_test, y_pred)
	mae = mean_absolute_error(y_test, y_pred)

	df_metrics.loc[type(model).__name__] = {
			'MSE':mse,
			'MAE':mae,
			'R2':r2
		}
	
	display(df_metrics)
	# print("Mean Squared Error:", mse)
	# print("Mean Absolute Error:", mae)
	# print("R-squared:", r2)

Regressão Linear

In [205]:
linear_model = LinearRegression().fit(X_train, y_train)

In [206]:
# Predictions baseado nos dados de teste
y_pred_linear = linear_model.predict(X_test)

In [207]:
create_metrics(linear_model, y_test, y_pred_linear)

,MSE,MAE,R2
LinearRegression,137682.733102,255.367493,0.810145


In [208]:
def plot_predict(y_test, y_predict):
  df_ = pd.DataFrame({'predict': y_predict, 'real': y_test})
  fig = px.scatter(df_, x='predict', y='real')
  range_axs = (0, df_.max().max())
  fig.update_xaxes(range=range_axs)
  fig.update_yaxes(range=range_axs)
  fig.show()

In [209]:
print('intercept = ', linear_model.intercept_)
# plotando nota real x nota predita
plot_predict(y_test, y_pred_linear)


intercept =  210.50773717894026


In [210]:
# Inicializando LassoCV com uma lista de alpha e cross-validation folds
lasso_cv = LassoCV(alphas=[0.01, 0.1, 1.0, 10.0], cv=10, random_state=4712389)
# Fit no modelo
lasso_model = lasso_cv.fit(X_train, y_train)

y_pred_lasso = lasso_model.predict(X_test)

# Melhor alpha
best_alpha = lasso_cv.alpha_
print("Best alpha:", best_alpha)

/home/valter/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 11188072.092190593, tolerance: 131211.3381234167

/home/valter/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 10909127.720227838, tolerance: 129751.65243003717

/home/valter/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 6194389.043857008, tolerance: 132610.9899420434

/home/valter/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning:

Objective did not converge. You might want t

Best alpha: 0.1


In [211]:
create_metrics(lasso_model, y_test, y_pred_lasso)
plot_predict(y_test, y_pred_lasso)

,MSE,MAE,R2
LinearRegression,137682.733102,255.367493,0.810145
LassoCV,136325.733416,253.622510,0.812017


Decision Tree

In [212]:
dt_model = DecisionTreeRegressor(random_state=4712389)  # You can adjust n_estimators and other hyperparameters

# Train the model on the training data
dt_model = dt_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_dt = dt_model.predict(X_test)

In [213]:
create_metrics(dt_model, y_test, y_pred_dt)
plot_predict(y_test, y_pred_dt)

,MSE,MAE,R2
LinearRegression,137682.733102,255.367493,0.810145
LassoCV,136325.733416,253.622510,0.812017
DecisionTreeRegressor,197586.775202,268.506577,0.727542


Random Forest

In [214]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=4712389)  # You can adjust n_estimators and other hyperparameters

# Train the model on the training data
rf_model = rf_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_rf = rf_model.predict(X_test)

In [215]:
create_metrics(rf_model, y_test, y_pred_rf)
plot_predict(y_test, y_pred_rf)

,MSE,MAE,R2
LinearRegression,137682.733102,255.367493,0.810145
LassoCV,136325.733416,253.622510,0.812017
DecisionTreeRegressor,197586.775202,268.506577,0.727542
RandomForestRegressor,143667.769170,230.525812,0.801892


SVR

In [216]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

rng = np.random.RandomState(0)
svr_model = SVR(kernel='linear', C=4.0, epsilon=0.2)
svr_model = svr_model.fit(X_train, y_train)

y_pred_svr = svr_model.predict(X_test)

In [217]:
create_metrics(svr_model, y_test, y_pred_svr)
plot_predict(y_test, y_pred_svr)

,MSE,MAE,R2
LinearRegression,137682.733102,255.367493,0.810145
LassoCV,136325.733416,253.622510,0.812017
DecisionTreeRegressor,197586.775202,268.506577,0.727542
RandomForestRegressor,143667.769170,230.525812,0.801892
SVR,240413.821105,320.730154,0.668487


XGBoost

In [218]:
xgb_model = xgb.XGBRegressor()

# otimização de hiperparametros
parameters = {
    'max_depth': [3, 7, 12],
    'min_child_weight': [5, 10, 20],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.75, 1],
    'n_estimators': [400],
    'eta':[0.01, 0.1, 0.5]
    }

xgb_model = GridSearchCV(
    xgb_model, 
    parameters, 
    n_jobs=4, 
    cv=5,
    scoring=make_scorer(mean_squared_error, greater_is_better=False)
)

xgb_model = xgb_model.fit(X_train, y_train, verbose=1)
y_pred_xgb = xgb_model.predict(X_test)

In [219]:
create_metrics(xgb_model, y_test, y_pred_xgb)
plot_predict(y_test, y_pred_xgb)

,MSE,MAE,R2
LinearRegression,137682.733102,255.367493,0.810145
LassoCV,136325.733416,253.622510,0.812017
DecisionTreeRegressor,197586.775202,268.506577,0.727542
RandomForestRegressor,143667.769170,230.525812,0.801892
SVR,240413.821105,320.730154,0.668487
GridSearchCV,127247.027467,225.097152,0.824536


In [221]:
df_metrics = df_metrics.rename(index={'GridSearchCV':'XGBoost'})
df_metrics

,MSE,MAE,R2
LinearRegression,137682.733102,255.367493,0.810145
LassoCV,136325.733416,253.622510,0.812017
DecisionTreeRegressor,197586.775202,268.506577,0.727542
RandomForestRegressor,143667.769170,230.525812,0.801892
SVR,240413.821105,320.730154,0.668487
XGBoost,127247.027467,225.097152,0.824536


XGBoost contém menor MSE, menor MAE e maior R2